# Imports, authentication and config.

In [ ]:
%load_ext google.cloud.bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import datetime
from datetime import datetime, timedelta
from datetime import date as dt
from functools import reduce

auth.authenticate_user()

creds, _ = default()
gc = gspread.authorize(creds)

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive_auth = GoogleDrive(gauth)

#Excel ID
CDS_Database = '1Pxi297bYvI7WekfU8LajzG2YQwCH4tiEV18um8dLjzM'

# Reading the data and filtering by media origin



In [ ]:
# Get Data from Google Sheets
sheet = gc.open_by_key(CDS_Database)  # ID of Google Sheets
worksheet = sheet.worksheet('Global')
data = worksheet.get_all_values()

# Convert into Pandas Dataframe
df = pd.DataFrame(data[1:], columns=data[0])

# Datetime Format of Date Column
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

df

,Date,Geography,Country_index,GeoRisk,EPU_Total,EPU_Foreign,EPU_Local,PoliticalTensions_Total,PoliticalTensions_Foreign,PoliticalTensions_Local,CDS,VIX,Fed_Funds
0,2018-01-01,Argentina,1,"1,293557645","0,2322128008","0,2322128008","0,2322128008","0,1469986637","0,1688325815","0,09330600924","238,22","10,4","1,33"
1,2018-01-02,Argentina,1,"1,013621658","0,03517369025","0,03517369025","0,03517369025","0,09320675676","0,1128841468","0,05601508418","234,89","9,77","1,42"
2,2018-01-03,Argentina,1,"0,7706328185","-0,1786035979","-0,1786035979","-0,1786035979","0,08874473842","0,1067740945","0,05481802426","235,065","9,15","1,42"
3,2018-01-04,Argentina,1,"0,4757909149","-0,4128786369","-0,4128786369","-0,4128786369","0,0934572973","0,1058251812","0,06179175258","232,275","9,22","1,42"
4,2018-01-05,Argentina,1,"0,3537483698","-0,5716739254","-0,5716739254","-0,5716739254","0,06470990857","0,06378060493","0,06691861635","235,545","9,22","1,42"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51325,2024-10-07,Venezuela,29,"0,02190624796","-1,352928443","-1,424608817","-1,046010805","-0,8822814317","-1,049657109","1,942829262","16599,18","22,64","4,83"
51326,2024-10-08,Venezuela,29,"-0,126427547","-1,343154434","-1,41536252","-1,029530487","-0,8663452077","-1,012949048","1,660730313","16599,18","21,42","4,83"
51327,2024-10-09,Venezuela,29,"-0,216925263","-1,295811241","-1,334035972","-1,193595478","-0,9882101026","-1,135096554","1,463969396","16599,18","20,86","4,83"
51328,2024-10-10,Venezuela,29,"-0,2492115477","-1,265344357","-1,29986425","-1,186378001","-1,018535227","-1,155434942","1,246757355","16599,18","20,93","4,83"


In [ ]:
#Function that dynamically selects columns according to the media source
def filter_by_media_source(df, media_type):
# Columns to select: those ending with "_mediatype" and columns not linked to media sources
  selected_columns = [col for col in df.columns if col.endswith(f"_{media_type}")
   or not any(suffix in col for suffix in ["_Total", "_Foreign", "_Local"])]
  return df[selected_columns]

#Create DataFrame for each media source
df_total = filter_by_media_source(df, "Total")
df_foreign = filter_by_media_source(df, "Foreign")
df_local = filter_by_media_source(df, "Local")

df_total

,Date,Geography,Country_index,GeoRisk,EPU_Total,PoliticalTensions_Total,CDS,VIX,Fed_Funds
0,2018-01-01,Argentina,1,"1,293557645","0,2322128008","0,1469986637","238,22","10,4","1,33"
1,2018-01-02,Argentina,1,"1,013621658","0,03517369025","0,09320675676","234,89","9,77","1,42"
2,2018-01-03,Argentina,1,"0,7706328185","-0,1786035979","0,08874473842","235,065","9,15","1,42"
3,2018-01-04,Argentina,1,"0,4757909149","-0,4128786369","0,0934572973","232,275","9,22","1,42"
4,2018-01-05,Argentina,1,"0,3537483698","-0,5716739254","0,06470990857","235,545","9,22","1,42"
...,...,...,...,...,...,...,...,...,...
51325,2024-10-07,Venezuela,29,"0,02190624796","-1,352928443","-0,8822814317","16599,18","22,64","4,83"
51326,2024-10-08,Venezuela,29,"-0,126427547","-1,343154434","-0,8663452077","16599,18","21,42","4,83"
51327,2024-10-09,Venezuela,29,"-0,216925263","-1,295811241","-0,9882101026","16599,18","20,86","4,83"
51328,2024-10-10,Venezuela,29,"-0,2492115477","-1,265344357","-1,018535227","16599,18","20,93","4,83"
